# 🔌 Advanced Extensibility

<div style="background-color: #e3f2fd; padding: 15px; border-radius: 5px; border-left: 5px solid #2196F3;">
<b>📓 Notebook Information</b><br>
<b>Level:</b> Advanced<br>
<b>Estimated Time:</b> 30 minutes<br>
<b>Prerequisites:</b> All basic and intermediate notebooks<br>
<b>Dataset:</b> Custom synthetic data
</div>

---

## 🎯 Learning Objectives

By the end of this notebook, you will be able to:
- ✅ Create custom validation tests
- ✅ Extend DeepBridge with custom metrics
- ✅ Build custom data transformers
- ✅ Implement custom report renderers
- ✅ Integrate external validation frameworks
- ✅ Design reusable validation components

---

## 📚 Table of Contents

1. [Introduction](#intro)
2. [Setup](#setup)
3. [Custom Validation Tests](#custom-tests)
4. [Custom Metrics](#custom-metrics)
5. [Custom Transformers](#transformers)
6. [Custom Report Renderers](#renderers)
7. [Plugin Architecture](#plugins)
8. [Best Practices](#practices)
9. [Conclusion](#conclusion)

<a id="intro"></a>
## 1. 📖 Introduction

### Why Extend DeepBridge?

**The Reality:**
- 🎯 **Domain-specific needs** - Every industry has unique requirements
- 📊 **Custom metrics** - Standard metrics don't capture everything
- 🔬 **Novel tests** - New research → new validation techniques
- 🏢 **Company standards** - Organization-specific validation policies
- 🔌 **Integration** - Connect with existing tools and workflows

**Real-world examples:**

1. **Medical AI**
   ```python
   # Custom test: Clinical safety margin
   class ClinicalSafetyTest(CustomTest):
       """Ensure model errs on the side of caution"""
       def run(self, dataset):
           # False negatives > false positives for cancer detection
           return validate_safety_margin(dataset)
   ```

2. **Finance**
   ```python
   # Custom metric: Economic value
   def profit_based_accuracy(y_true, y_pred, loan_amounts):
       """Weight accuracy by loan size"""
       return calculate_expected_profit(y_true, y_pred, loan_amounts)
   ```

3. **E-commerce**
   ```python
   # Custom transformer: Seasonality injection
   class SeasonalTransformer:
       """Test resilience to seasonal patterns"""
       def transform(self, X):
           return inject_seasonal_patterns(X)
   ```

### DeepBridge Extension Points

```
DeepBridge Core
├── 1. Custom Tests
│   └── Inherit from BaseTest
├── 2. Custom Metrics  
│   └── Register with MetricRegistry
├── 3. Custom Transformers
│   └── Implement transform() method
├── 4. Custom Renderers
│   └── Inherit from BaseRenderer
└── 5. Plugins
    └── Package as installable module
```

### Extension Philosophy

| Principle | Description | Example |
|-----------|-------------|----------|
| **Open/Closed** | Open for extension, closed for modification | Add tests without changing core |
| **Composability** | Combine components like LEGO | Chain transformers |
| **Discoverability** | Easy to find and use extensions | Auto-register plugins |
| **Consistency** | Follow same API patterns | Same interface as built-in tests |

**Let's build custom components!** 🔌

<a id="setup"></a>
## 2. 🛠️ Setup

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from abc import ABC, abstractmethod
from typing import Dict, Any, Optional, List
from dataclasses import dataclass

# sklearn
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score

# DeepBridge
from deepbridge import DBDataset, Experiment

# Settings
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('Set2')
%matplotlib inline

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print("✅ Setup complete!")
print("🔌 Topic: Advanced Extensibility")

<a id="custom-tests"></a>
## 3. 🧪 Custom Validation Tests

### Define Base Test Interface

In [ ]:
print("🧪 Creating Custom Validation Test Framework\n")

# Base class for custom tests
class CustomValidationTest(ABC):
    """Base class for custom validation tests."""
    
    def __init__(self, name: str, description: str):
        self.name = name
        self.description = description
        self.results = {}
    
    @abstractmethod
    def run(self, dataset: DBDataset) -> Dict[str, Any]:
        """Run the validation test.
        
        Args:
            dataset: DBDataset to validate
            
        Returns:
            Dictionary with test results
        """
        pass
    
    def get_status(self, results: Dict[str, Any]) -> str:
        """Determine test status (PASS/FAIL/WARNING)."""
        if 'status' in results:
            return results['status']
        return 'UNKNOWN'
    
    def report(self) -> str:
        """Generate human-readable report."""
        status = self.get_status(self.results)
        return f"{self.name}: {status}"

print("✅ Base test interface defined")
print("\n📋 Required methods:")
print("   • run(dataset) → results")
print("   • get_status(results) → PASS/FAIL/WARNING")
print("   • report() → human-readable summary")

### Example: Business Logic Validation Test

In [ ]:
print("🔧 Creating Business Logic Validation Test\n")

class BusinessLogicTest(CustomValidationTest):
    """Validate model predictions against business rules."""
    
    def __init__(self, rules: Dict[str, Any]):
        super().__init__(
            name="Business Logic Validation",
            description="Ensure model respects business constraints"
        )
        self.rules = rules
    
    def run(self, dataset: DBDataset) -> Dict[str, Any]:
        """Run business logic validation."""
        results = {
            'test_name': self.name,
            'violations': [],
            'rules_checked': len(self.rules),
            'rules_passed': 0
        }
        
        # Get predictions
        y_pred = dataset.model.predict(dataset.X_test)
        y_proba = dataset.model.predict_proba(dataset.X_test)
        
        # Rule 1: Minimum confidence threshold
        if 'min_confidence' in self.rules:
            threshold = self.rules['min_confidence']
            max_probs = y_proba.max(axis=1)
            low_confidence = (max_probs < threshold).sum()
            
            if low_confidence > 0:
                results['violations'].append({
                    'rule': 'Minimum Confidence',
                    'threshold': threshold,
                    'violations': int(low_confidence),
                    'percentage': float(low_confidence / len(y_pred) * 100)
                })
            else:
                results['rules_passed'] += 1
        
        # Rule 2: Maximum false negative rate (for safety-critical apps)
        if 'max_fnr' in self.rules:
            from sklearn.metrics import confusion_matrix
            cm = confusion_matrix(dataset.y_test, y_pred)
            fn = cm[1, 0] if cm.shape[0] > 1 else 0
            tp = cm[1, 1] if cm.shape[0] > 1 else 0
            fnr = fn / (fn + tp) if (fn + tp) > 0 else 0
            
            max_fnr = self.rules['max_fnr']
            if fnr > max_fnr:
                results['violations'].append({
                    'rule': 'Maximum False Negative Rate',
                    'threshold': max_fnr,
                    'actual': float(fnr),
                    'severity': 'CRITICAL'
                })
            else:
                results['rules_passed'] += 1
        
        # Rule 3: Class distribution limits
        if 'class_distribution' in self.rules:
            pred_dist = np.bincount(y_pred) / len(y_pred)
            expected = self.rules['class_distribution']
            
            for cls, (min_pct, max_pct) in expected.items():
                actual_pct = pred_dist[cls] if cls < len(pred_dist) else 0
                if not (min_pct <= actual_pct <= max_pct):
                    results['violations'].append({
                        'rule': f'Class {cls} Distribution',
                        'expected_range': f'{min_pct:.1%} - {max_pct:.1%}',
                        'actual': f'{actual_pct:.1%}'
                    })
                else:
                    results['rules_passed'] += 1
        
        # Determine status
        if len(results['violations']) == 0:
            results['status'] = 'PASS'
        elif any(v.get('severity') == 'CRITICAL' for v in results['violations']):
            results['status'] = 'FAIL'
        else:
            results['status'] = 'WARNING'
        
        self.results = results
        return results

print("✅ Business Logic Test created")
print("\n📋 Validated rules:")
print("   • Minimum prediction confidence")
print("   • Maximum false negative rate (safety-critical)")
print("   • Class distribution constraints")

### Run Custom Test

In [ ]:
print("🔬 Running Custom Business Logic Test\n")

# Create dataset
X, y = make_classification(
    n_samples=1000,
    n_features=20,
    n_informative=15,
    n_classes=2,
    random_state=RANDOM_STATE
)

df = pd.DataFrame(X, columns=[f'feature_{i}' for i in range(X.shape[1])])
df['target'] = y

# Train model
X_train, X_test, y_train, y_test = train_test_split(
    df.drop('target', axis=1), df['target'],
    test_size=0.2, random_state=RANDOM_STATE
)

model = RandomForestClassifier(n_estimators=100, random_state=RANDOM_STATE)
model.fit(X_train, y_train)

# Create DBDataset
dataset = DBDataset(
    data=df,
    target_column='target',
    model=model,
    test_size=0.2,
    random_state=RANDOM_STATE
)

# Define business rules
business_rules = {
    'min_confidence': 0.7,  # Predictions must be 70%+ confident
    'max_fnr': 0.15,  # False negative rate < 15%
    'class_distribution': {
        0: (0.3, 0.7),  # Class 0: 30-70%
        1: (0.3, 0.7)   # Class 1: 30-70%
    }
}

# Run test
test = BusinessLogicTest(rules=business_rules)
results = test.run(dataset)

print("=" * 70)
print(f"Test: {results['test_name']}")
print(f"Status: {results['status']}")
print(f"Rules Checked: {results['rules_checked']}")
print(f"Rules Passed: {results['rules_passed']}")

if results['violations']:
    print(f"\n⚠️  Violations ({len(results['violations'])}):")
    for v in results['violations']:
        print(f"\n   Rule: {v['rule']}")
        for key, value in v.items():
            if key != 'rule':
                print(f"      {key}: {value}")
else:
    print(f"\n✅ All business rules satisfied!")

print("\n💡 This test ensures model behavior aligns with business requirements!")

<a id="custom-metrics"></a>
## 4. 📊 Custom Metrics

### Define Domain-Specific Metrics

In [ ]:
print("📊 Creating Custom Metrics\n")

# Custom Metric 1: Cost-sensitive accuracy
def cost_weighted_accuracy(y_true, y_pred, costs: Dict[str, float]) -> float:
    """Calculate accuracy weighted by misclassification costs.
    
    Args:
        y_true: True labels
        y_pred: Predicted labels
        costs: Dictionary of costs {'fn': cost, 'fp': cost}
    
    Returns:
        Cost-weighted accuracy score
    """
    from sklearn.metrics import confusion_matrix
    
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel() if cm.size == 4 else (0, 0, 0, 0)
    
    # Total cost of errors
    total_cost = fn * costs.get('fn', 1.0) + fp * costs.get('fp', 1.0)
    
    # Maximum possible cost (all predictions wrong)
    max_cost = len(y_true) * max(costs.get('fn', 1.0), costs.get('fp', 1.0))
    
    # Cost-weighted accuracy (1 = perfect, 0 = worst)
    return 1 - (total_cost / max_cost) if max_cost > 0 else 1.0


# Custom Metric 2: Balanced performance score
def balanced_performance_score(y_true, y_pred, weights: Dict[str, float]) -> float:
    """Composite score combining multiple metrics.
    
    Args:
        y_true: True labels
        y_pred: Predicted labels  
        weights: Metric weights {'accuracy': w, 'precision': w, 'recall': w}
    
    Returns:
        Weighted composite score
    """
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, average='weighted', zero_division=0)
    rec = recall_score(y_true, y_pred, average='weighted', zero_division=0)
    
    score = (
        weights.get('accuracy', 1.0) * acc +
        weights.get('precision', 1.0) * prec +
        weights.get('recall', 1.0) * rec
    )
    
    total_weight = sum(weights.values())
    return score / total_weight if total_weight > 0 else 0.0


# Custom Metric 3: Confidence-calibrated accuracy
def confidence_calibrated_accuracy(y_true, y_pred, y_proba) -> float:
    """Accuracy adjusted for prediction confidence.
    
    Rewards high-confidence correct predictions,
    penalizes high-confidence incorrect predictions.
    """
    max_proba = y_proba.max(axis=1)
    correct = (y_true == y_pred).astype(float)
    
    # Weight by confidence: correct predictions gain, wrong lose
    weighted_correct = correct * max_proba - (1 - correct) * max_proba
    
    # Normalize to [0, 1]
    return (weighted_correct.mean() + 1) / 2


print("✅ Custom metrics defined:")
print("\n1. Cost-Weighted Accuracy")
print("   • Accounts for different error costs")
print("   • Use case: Fraud detection, medical diagnosis")
print("\n2. Balanced Performance Score")
print("   • Composite of accuracy, precision, recall")
print("   • Configurable weights for each metric")
print("\n3. Confidence-Calibrated Accuracy")
print("   • Penalizes overconfident wrong predictions")
print("   • Rewards calibrated confidence")

### Test Custom Metrics

In [ ]:
print("🧪 Testing Custom Metrics\n")

# Get predictions
y_true = dataset.y_test
y_pred = model.predict(dataset.X_test)
y_proba = model.predict_proba(dataset.X_test)

# Standard metrics
acc = accuracy_score(y_true, y_pred)
prec = precision_score(y_true, y_pred, average='weighted')
rec = recall_score(y_true, y_pred, average='weighted')

# Custom metrics
cost_acc = cost_weighted_accuracy(y_true, y_pred, costs={'fn': 10, 'fp': 1})
balanced = balanced_performance_score(
    y_true, y_pred, 
    weights={'accuracy': 0.3, 'precision': 0.3, 'recall': 0.4}
)
conf_acc = confidence_calibrated_accuracy(y_true, y_pred, y_proba)

# Display results
results_df = pd.DataFrame({
    'Metric': [
        'Standard Accuracy',
        'Standard Precision',
        'Standard Recall',
        '---',
        'Cost-Weighted Accuracy',
        'Balanced Performance Score',
        'Confidence-Calibrated Accuracy'
    ],
    'Score': [
        f'{acc:.4f}',
        f'{prec:.4f}',
        f'{rec:.4f}',
        '---',
        f'{cost_acc:.4f}',
        f'{balanced:.4f}',
        f'{conf_acc:.4f}'
    ],
    'Type': [
        'Standard', 'Standard', 'Standard', '',
        'Custom', 'Custom', 'Custom'
    ]
})

print("📊 Metrics Comparison:\n")
display(results_df.style.apply(
    lambda x: ['background-color: #e3f2fd' if v == 'Custom' 
               else 'background-color: #f5f5f5' if v == 'Standard'
               else '' for v in x],
    subset=['Type']
).hide(axis='index'))

print("\n💡 Custom metrics provide domain-specific insights!")
print(f"   Cost-weighted accuracy is lower ({cost_acc:.3f} vs {acc:.3f})")
print(f"   because false negatives are 10x more expensive than false positives.")

<a id="transformers"></a>
## 5. 🔄 Custom Transformers

### Build Data Transformation Components

In [ ]:
print("🔄 Creating Custom Data Transformers\n")

class BusinessHoursTransformer:
    """Transform data to simulate business hours patterns."""
    
    def __init__(self, feature_cols: List[str], scale_factor: float = 0.3):
        self.feature_cols = feature_cols
        self.scale_factor = scale_factor
    
    def transform(self, X: pd.DataFrame) -> pd.DataFrame:
        """Simulate business hours by scaling certain features."""
        X_transformed = X.copy()
        
        for col in self.feature_cols:
            if col in X_transformed.columns:
                # Add temporal pattern (sinusoidal)
                pattern = np.sin(np.linspace(0, 2*np.pi, len(X_transformed)))
                X_transformed[col] = X_transformed[col] * (1 + self.scale_factor * pattern)
        
        return X_transformed


class OutlierInjectionTransformer:
    """Inject outliers to test model robustness."""
    
    def __init__(self, outlier_fraction: float = 0.05, std_multiplier: float = 3.0):
        self.outlier_fraction = outlier_fraction
        self.std_multiplier = std_multiplier
    
    def transform(self, X: pd.DataFrame) -> pd.DataFrame:
        """Inject outliers into random samples."""
        X_transformed = X.copy()
        n_outliers = int(len(X) * self.outlier_fraction)
        
        # Select random samples to corrupt
        outlier_idx = np.random.choice(len(X), n_outliers, replace=False)
        
        # Inject outliers (shift by multiple of std)
        for col in X_transformed.columns:
            if X_transformed[col].dtype in ['float64', 'float32', 'int64', 'int32']:
                std = X_transformed[col].std()
                X_transformed.loc[outlier_idx, col] += np.random.choice(
                    [-1, 1], size=n_outliers
                ) * self.std_multiplier * std
        
        return X_transformed


class MissingDataTransformer:
    """Simulate missing data patterns."""
    
    def __init__(self, missing_fraction: float = 0.1, pattern: str = 'random'):
        self.missing_fraction = missing_fraction
        self.pattern = pattern  # 'random', 'mcar', 'mar', 'mnar'
    
    def transform(self, X: pd.DataFrame) -> pd.DataFrame:
        """Inject missing values."""
        X_transformed = X.copy()
        n_values = int(X.size * self.missing_fraction)
        
        if self.pattern == 'random':
            # Randomly select values to make NaN
            rows = np.random.choice(len(X), n_values)
            cols = np.random.choice(X.columns, n_values)
            
            for row, col in zip(rows, cols):
                X_transformed.loc[row, col] = np.nan
        
        return X_transformed


print("✅ Custom transformers created:")
print("\n1. BusinessHoursTransformer")
print("   • Simulates temporal patterns")
print("   • Tests model on time-varying data")
print("\n2. OutlierInjectionTransformer")
print("   • Injects statistical outliers")
print("   • Tests robustness to anomalies")
print("\n3. MissingDataTransformer")
print("   • Simulates missing values")
print("   • Tests handling of incomplete data")

### Apply Transformers

In [ ]:
print("🧪 Testing Custom Transformers\n")

# Original performance
y_pred_original = model.predict(dataset.X_test)
acc_original = accuracy_score(dataset.y_test, y_pred_original)

# Apply transformers
transformers = [
    ('Business Hours', BusinessHoursTransformer(feature_cols=['feature_0', 'feature_1'])),
    ('Outlier Injection', OutlierInjectionTransformer(outlier_fraction=0.05)),
    ('Missing Data', MissingDataTransformer(missing_fraction=0.1))
]

results = [('Original', acc_original)]

for name, transformer in transformers:
    X_transformed = transformer.transform(dataset.X_test)
    
    # Handle missing values if any
    if X_transformed.isna().any().any():
        X_transformed = X_transformed.fillna(X_transformed.mean())
    
    # Test model on transformed data
    y_pred = model.predict(X_transformed)
    acc = accuracy_score(dataset.y_test, y_pred)
    results.append((name, acc))

# Display results
print("📊 Model Performance Under Different Transformations:\n")
print("=" * 70)

for name, acc in results:
    degradation = ((acc_original - acc) / acc_original * 100) if name != 'Original' else 0
    status = '✅' if degradation < 5 else '⚠️' if degradation < 10 else '❌'
    
    print(f"{status} {name:20s}: {acc:.4f} ({degradation:+.1f}% degradation)")

print("\n💡 Transformers help test model resilience to real-world conditions!")

<a id="practices"></a>
## 8. ✨ Best Practices

### Extension Development Guidelines

<div style="background-color: #e8f5e9; padding: 15px; border-radius: 5px; border-left: 5px solid #4CAF50;">
<b>✅ DO</b><br><br>

1. **Follow Standard Interfaces**
   ```python
   # ✅ GOOD: Consistent with sklearn API
   class MyTransformer:
       def fit(self, X, y=None):
           return self
       
       def transform(self, X):
           return X_transformed
   ```

2. **Document Thoroughly**
   - Clear docstrings
   - Usage examples
   - Parameter descriptions
   - Expected inputs/outputs

3. **Include Tests**
   ```python
   # Always test your extensions
   def test_custom_metric():
       y_true = [0, 1, 1, 0]
       y_pred = [0, 1, 0, 0]
       score = my_custom_metric(y_true, y_pred)
       assert 0 <= score <= 1
   ```

4. **Handle Edge Cases**
   - Empty inputs
   - Single class
   - Missing values
   - Invalid parameters

5. **Make It Configurable**
   ```python
   # ✅ GOOD: Configurable behavior
   class MyTest:
       def __init__(self, threshold=0.8, mode='strict'):
           self.threshold = threshold
           self.mode = mode
   ```

</div>

<div style="background-color: #ffebee; padding: 15px; border-radius: 5px; border-left: 5px solid #f44336; margin-top: 15px;">
<b>❌ DON'T</b><br><br>

1. **Modify Core Classes**
   ```python
   # ❌ BAD: Modifying DeepBridge internals
   from deepbridge import Experiment
   Experiment.some_method = my_custom_method
   ```
   ```python
   # ✅ GOOD: Extend, don't modify
   class MyExperiment(Experiment):
       def custom_method(self):
           pass
   ```

2. **Ignore Performance**
   - Profile your extensions
   - Optimize hot paths
   - Consider memory usage

3. **Hardcode Values**
   ```python
   # ❌ BAD: Hardcoded
   threshold = 0.75
   ```
   ```python
   # ✅ GOOD: Configurable
   def __init__(self, threshold=0.75):
       self.threshold = threshold
   ```

4. **Assume Data Format**
   - Validate inputs
   - Handle different dtypes
   - Support various shapes

</div>

<a id="conclusion"></a>
## 9. 🎓 Conclusion

### What You Learned

- ✅ **Custom tests** - Build domain-specific validation tests
- ✅ **Custom metrics** - Create business-relevant metrics
- ✅ **Custom transformers** - Test resilience to perturbations
- ✅ **Extension patterns** - Follow best practices
- ✅ **Integration** - Connect with existing workflows

### Key Takeaways

1. 🔌 **Extend, don't modify** - Use inheritance and composition
2. 📋 **Follow interfaces** - Consistency with sklearn/DeepBridge APIs
3. 📝 **Document everything** - Your future self will thank you
4. 🧪 **Test thoroughly** - Extensions need tests too
5. ⚙️ **Make configurable** - Flexibility is key
6. 🎯 **Solve real problems** - Build what you actually need

### Extension Use Cases by Industry

| Industry | Custom Extensions | Example |
|----------|------------------|----------|
| **Healthcare** | Safety-critical tests | Max false negative rate test |
| **Finance** | Economic metrics | Profit-weighted accuracy |
| **E-commerce** | Business logic tests | Price range validation |
| **Manufacturing** | Quality assurance | Defect rate constraints |
| **Insurance** | Regulatory compliance | Actuarial fairness tests |

### Building Your Extension Library

```python
# 1. Start simple
class MyFirstTest(CustomValidationTest):
    def run(self, dataset):
        # Your validation logic
        return results

# 2. Test it
test = MyFirstTest()
results = test.run(dataset)

# 3. Package for reuse
# my_validations/
#   __init__.py
#   tests.py
#   metrics.py
#   transformers.py

# 4. Share with team
from my_validations import MyFirstTest
```

### Real-World Extension Example

**Medical AI Safety Suite:**
```python
from deepbridge import Experiment
from medical_ai_validation import (
    ClinicalSafetyTest,        # Max FN rate
    CalibrationTest,            # Probability calibration
    SubgroupFairnessTest,      # Performance across demographics
    RegulatoryComplianceTest   # FDA requirements
)

# Run complete medical validation suite
exp = Experiment(dataset, experiment_type='binary_classification')

# Standard tests
exp.run_test('robustness')
exp.run_test('uncertainty')

# Custom medical tests
clinical_safety = ClinicalSafetyTest(max_fnr=0.05)
clinical_safety.run(exp.dataset)

calibration = CalibrationTest()
calibration.run(exp.dataset)

# Generate FDA-compliant report
exp.generate_report(
    'fda_submission_report.pdf',
    include_custom_tests=[clinical_safety, calibration]
)
```

### Next Steps

1. **Identify your needs** - What validations does your domain require?
2. **Build incrementally** - Start with one custom test
3. **Test extensively** - Ensure reliability
4. **Share and iterate** - Get feedback from team
5. **Contribute back** - Open source useful extensions

### Resources

- 📚 **DeepBridge Extension Guide**: docs/extending-deepbridge.md
- 🔌 **Example Extensions**: examples/custom_extensions/
- 💬 **Community Forum**: discuss.deepbridge.ai
- 🐛 **Report Issues**: github.com/deepbridge/deepbridge/issues

---

**Remember: The best validation framework is the one you can extend to meet your needs!** 🔌✨

**You've completed all 27 DeepBridge notebooks! 🎉**